# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355142361812                   -1.10    6.1    183ms
  2   -9.356770821902       -2.79       -1.43    1.0   80.5ms
  3   -9.357069077521       -3.53       -2.77    2.5    101ms
  4   -9.357119390622       -4.30       -2.99    6.6    224ms
  5   -9.357119835129       -6.35       -3.13    2.4   95.1ms
  6   -9.357120082871       -6.61       -3.28    1.0   76.9ms
  7   -9.357120235184       -6.82       -3.45    1.0    146ms
  8   -9.357120290197       -7.26       -3.62    1.1   93.5ms
  9   -9.357120315548       -7.60       -3.88    1.1    371ms
 10   -9.357120321781       -8.21       -4.12    2.5    100ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74245481967                   -0.97    6.4    545ms
  2   -18.78141643024       -1.41       -1.34    1.6    240ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      2
│     13
│      4
│     12
│      2
│      9
│      1
│      5
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -18.77986563419   +   -2.81       -1.82    6.0    469ms
  4   -18.79154501017       -1.93       -1.93    3.4    319ms
  5   -18.79233461034       -3.10       -2.13    2.2    270ms
  6   -18.79253495215       -3.70       -2.36    1.0    234ms
  7   -18.79261231402       -4.11       -2.88    1.2    272ms
  8   -18.79261689194       -5.34       -3.01    3.9    362ms
  9   -18.79261853191       -5.79       -3.08    1.1    228ms
 10   -18.79262179602       -5.49       -3.33    1.0    217ms
 11   -18.79262346274       -5.78    

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.53535450304                   -0.84   10.1    1.93s
  2   -37.55647998096       -1.68       -1.20    2.6    807ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     17
│      6
│     14
│     20
│     27
│     19
│     13
│     10
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -37.56463145686       -2.09       -2.15   15.8    1.60s
  4   -37.56493883233       -3.51       -2.44   10.9    1.89s
  5   -37.56498172110       -4.37       -3.43    2.1    886ms
  6   -37.56498480400       -5.51       -3.79    9.8    1.88s
  7   -37.56498517436       -6.43       -4.29    3.0    1.03s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355191238258                   -1.10    6.1    186ms
  2   -9.356820603236       -2.79       -1.90    1.0   70.5ms
  3   -9.357054884265       -3.63       -2.38    5.5    160ms
  4   -9.357104775539       -4.30       -2.64    3.0    108ms
  5   -9.357119852788       -4.82       -3.60    1.0   73.1ms
  6   -9.357120221554       -6.43       -3.80    5.2    181ms
  7   -9.357120305634       -7.08       -4.15    1.6   86.6ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     11
│      3
│     17
│      4
│     16
│     11
│     14
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54156007854                   -0.84    9.8    1.82s
  2   -37.48819201771   +   -1.27       -1.41    2.9    710ms
  3   +8.061082873936   +    1.66       -0.23    8.5    1.93s
  4   -35.44792376269        1.64       -0.86    8.1    2.28s
  5   -37.40314387059        0.29       -1.34    8.0    1.43s
  6   -32.78088370663   +    0.66       -0.73    6.0    1.29s
  7   -37.53702704973        0.68       -1.74    6.5    1.34s
  8   -37.54965824856       -1.90       -1.77    2.1    1.16s
  9   -37.55434095490       -2.33       -2.00    1.6    755ms
 10   -37.56303272501       -2.06       -2.30    1.9    824ms
 11   -37.56484533300       -2.74    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.